In [ ]:
# Luồng camera để truyền frame ảnh lên cho YOLO detect <phía producer id = 1>
import cv2
from kafka import KafkaProducer

bootstrap_servers = 'localhost:9092'
topic = 'video_frames'
producer1 = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Dùng OpenCV để phân tách các frame ảnh 
video_path = 'VideoSend_0.mkv'
cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển frame ảnh thành kiểu byte (đảm bảo đúng kiểu dữ liệu tham số đầu vào)
    _, img_encoded = cv2.imencode('.png', frame)
    img_bytes = img_encoded.tobytes()

    # Dùng producer send ảnh đến topic Kafka
    producer1.send(topic, value=img_bytes)

cap.release()

In [ ]:
# Luồng camera để truyền frame ảnh lên cho YOLO detect <phía consumer id = 1>
import cv2
import numpy as np
from kafka import KafkaConsumer

# Khởi tạo consumer
bootstrap_servers = 'localhost:9092'
topic = 'video_frames'
consumer1 = KafkaConsumer(topic, bootstrap_servers=bootstrap_servers)

# Tạo cửa số nhờ OpenCV để hiện thị các frame ảnh
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

for msg in consumer1:
    img_bytes = msg.value

    # Convert bytes to image
    nparr = np.frombuffer(img_bytes, dtype=np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    # frame sẽ là một mảng NumPy có kiểu dữ liệu là numpy.ndarray và có thể được truy cập thông qua các chỉ số mảng để lấy các giá trị pixel.

    # Display frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [1]:
from PIL import Image
image = Image.open("0.jpg")

from ultralytics import YOLO
# Load the YOLOv8 model
yolo_model = YOLO('yolov8x.pt')
predict = yolo_model.predict(source=image)
bounding_boxes = predict[0].boxes.xywh


0: 448x640 12 persons, 3076.7ms
Speed: 7.0ms preprocess, 3076.7ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
